In [1]:
!pip -q install jieba

In [2]:
import re
import pickle
import unicodedata

In [3]:
from time import time
from itertools import chain
from itertools import zip_longest
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from mih import VocabularyBPE
from mih import TokenizerBPE
# from preprocess_v2 import NormalizerSGZChat
from preprocess_v3 import NormalizerSGZChat

2021-04-12 11:07:11 DEBUG Starting new HTTPS connection (1): huggingface.co:443
2021-04-12 11:07:12 DEBUG https://huggingface.co:443 "HEAD /xlm-roberta-base/resolve/main/config.json HTTP/1.1" 200 0
2021-04-12 11:07:12 DEBUG Starting new HTTPS connection (1): huggingface.co:443
2021-04-12 11:07:13 DEBUG https://huggingface.co:443 "HEAD /xlm-roberta-base/resolve/main/sentencepiece.bpe.model HTTP/1.1" 200 0
2021-04-12 11:07:13 DEBUG Starting new HTTPS connection (1): huggingface.co:443
2021-04-12 11:07:13 DEBUG https://huggingface.co:443 "HEAD /xlm-roberta-base/resolve/main/tokenizer.json HTTP/1.1" 200 0
Building model ...2021-04-12 11:07:14 DEBUG Starting new HTTPS connection (1): huggingface.co:443
2021-04-12 11:07:15 DEBUG https://huggingface.co:443 "HEAD /xlm-roberta-base/resolve/main/config.json HTTP/1.1" 200 0
2021-04-12 11:07:15 DEBUG Starting new HTTPS connection (1): huggingface.co:443
2021-04-12 11:07:16 DEBUG https://huggingface.co:443 "HEAD /xlm-roberta-base/resolve/main/confi

In [5]:
%run dataset.ipynb

(20000, 2)
(14686, 2)
34686 documents - 1.595MB (training set)
1017 documents - 0.061MB (test set)

Stored 'data_train' (DataFrame)
Stored 'X_train' (Series)
Stored 'X_test' (Series)
Stored 'y_train' (list)
Stored 'y_test' (list)
Stored 'data_train_size_mb' (float)


## Load

In [6]:
%store -r data_train
%store -r X_train
%store -r data_train_size_mb

## Preprocess

In [7]:
def is_special(i):
    
    is_1 =  i in ['<unk>', '<loc>', '<contact>', '<recruit>', '<corpus>', '<colonel>']
    is_2 = bool(re.match(r'^<num-[0-9]+>$', i))
    
    return is_1 or is_2

In [8]:
def not_special(i):
    return not is_special(i)

### replace

In [9]:
def __replace(s):
    if is_special(s): return s
    
    # 微信
    s = s.replace('威', '微')
    s = s.replace('徽', '微')
    s = s.replace('徵', '微')
    s = s.replace('亻言', '信')
    
    s = s.replace('微新', '微信')
    s = s.replace('微信', '微')
    
    # 加
    s = s.replace('咖', '加')
    s = s.replace('架', '加')
    s = s.replace('嫁', '加')
    s = s.replace('十', '加')
    s = s.replace('茄', '加')
    s = s.replace('迦', '加')
    
    s = s.replace('加下', '加')
    s = s.replace('加一下', '加')
    
    s = s.replace('加', '+')
    
    # 收人
    s = s.replace('活人', '人')
    # s = s.replace('收人', '<recruit>')
    
    # 团长
    s = s.replace('圕', '团')
    
    # 充
    s = s.replace('冲', '充')
    s = s.replace('直充', '充')
    
    # 出
    s = s.replace('础', '出')

    # 卖
    s = s.replace('麦', '出')
    
    return s

In [10]:
def replace(x):
    return [__replace(i) for i in x]

### split util

In [11]:
def split_regex(s, reg, flag):
    r = re.split(reg, s)
    if 1 >= len(r): return r
    r = list(chain.from_iterable(zip_longest(r[:-1], [], fillvalue=flag))) + r[-1:]
    return [i for i in r if i]

#### split location

In [12]:
def split_location(s):
    return split_regex(s, r'{localization:[0-9]+\-[0-9]+}', '<loc>')

#### split terminology

In [13]:
def __split_terminology(s, term, flag):
    if is_special(s): return [s]
    return split_regex(s, r'%s' % term, flag)

In [14]:
def split_terminology(x, term, flag):
    return list(chain.from_iterable([__split_terminology(i, term, flag) for i in x]))

#### split coordinates

In [15]:
# TODO:

#### split num + char

In [16]:
def __split_charnum(s):
    if is_special(s): return [s]
    return [c for c in re.split(r'([0-9a-z]+)', s) if c]

In [17]:
def split_charnum(x):
    return list(chain.from_iterable([__split_charnum(s) for s in x]))

#### convert num

In [18]:
def is_numeric(s):
    
    has_num = bool(re.findall(r'[0-9]+', s))
    hasnot_other = not bool(re.findall(r'[^0-9]+', s))
    
    return has_num and hasnot_other

In [19]:
def convert_num(x):
    """ tool
    """
    
    # return ['<num-%s>' % len(i) if i.isnumeric() else i for i in x]
    return ['<num-%s>' % len(i) if not_special(i) and is_numeric(i) else i for i in x]

#### convert num + char

In [20]:
def is_charnum(s):
    
    has_num = bool(re.findall(r'[0-9]+', s))
    has_char = bool(re.findall(r'[a-z]+', s))
    hasnot_other = not bool(re.findall(r'[^a-z0-9]+', s))
    
    return has_num and has_char and hasnot_other

In [21]:
def is_v_num(s):
    return bool(re.match(r'v[0-9]+', s))

In [22]:
def is_vx_num(s):
    return bool(re.match(r'vx[0-9]+', s))

In [23]:
def is_qq_num(s):
    return bool(re.match(r'qq[0-9]+', s))

In [24]:
def __convert_chars_numbers(s):
    
    if is_special(s): return [s]
    
    if is_v_num(s): return ['微', '<contact>']
    if is_vx_num(s): return ['微', '<contact>']
    if is_qq_num(s): return ['微', '<contact>']
    
    if is_charnum(s): return ['<contact>']
    
    return [s]
    

In [25]:
def convert_chars_numbers(x):
    # return ['<contact>' if not_special(i) and is_charnum(i) else i for i in x]
    return list(chain.from_iterable([__convert_chars_numbers(s) for s in x]))

#### split char, num, chinese + special

In [26]:
def __split_normal_special(s):
    if is_special(s): return s, ''
    
    # TODO: wheather , . ，。blank should be in valid
    return ''.join(re.findall(r'[,\+a-z0-9\u4e00-\u9fa5]+', s)), ''.join(re.findall(r'[^,\+a-z0-9\u4e00-\u9fa5]+', s))

In [27]:
def split_normal_special(x):
    
    r = [__split_normal_special(i) for i in x]
    
    r1, r2 = zip(*r)
    r1 = [i for i in r1 if i]
    r2 = [i for i in r2 if i]
    
    return r1, r2

#### split naive

In [28]:
def split_naive(s):
    # return jieba.lcut(s)
    # return jieba.lcut(s, cut_all=True)
    # return jieba.lcut_for_search(s)
    return list(s)
    
    # return jieba.lcut(s, cut_all=True) + list(s)
    # return jieba.lcut_for_search(s) + list(s)

#### split once

In [29]:
def __split_once(s):
    if is_special(s): return [s]
    
    # s1, s2 = split_normal_special(s)
    
    # return __split0(s1) + __split0(s2)
    # return split_naive(s1) + list(s2)
    
    # r = split_naive(s1) + list(s)
    
    # r1 = split_naive(s1)
    # r2 = list(s)
    # r = r1 + list(set(r2)-set(r1))
    
    # s = replace(s)
    # r = list(s)
    r = split_naive(s)
    
    return r

In [30]:
def split_once(x):
    return list(chain.from_iterable([__split_once(s) for s in x]))

#### stop words

In [31]:
stopwords1_usual = ['你', '我', '他', '她', '它', '们',
                    '吧', '吗', '嘛', '啊', '阿', '呢', '呀',
                    '的', '地', 
                    '怎', '么',
                    '那', '哪',
                    '就', '没', '了', '谢', '配', '合']

In [32]:
stopwords1 = stopwords1_usual

In [33]:
stopwords = stopwords1

In [34]:
def filter_stopwords(x):
    return [i for i in x if i not in stopwords]

### split 1

In [35]:
def split1(s):
    # preprocess
    s = s.replace(' ', '')    # TODO: maybe all blank
    s = s.lower()
    s = unicodedata.normalize('NFKC', s)
    
    # formated
    tokens = split_location(s)
    
    # normal and special-char
    tokens, tokens_special = split_normal_special(tokens)
    
    # user defined
    tokens = split_charnum(tokens)
    tokens = convert_num(tokens)
    tokens = convert_chars_numbers(tokens)
    
    # link
    tokens = replace(tokens)
    
    # split
    tokens = split_terminology(tokens, '收人', '<recruit>')
    tokens = split_terminology(tokens, '军团', '<corpus>')
    tokens = split_terminology(tokens, '团长', '<colonel>')
    tokens = split_once(tokens)
    
    # filter
    tokens = filter_stopwords(tokens)
    
    # merge tokens_special
    # tokens += list(''.join(tokens_special))
    tokens += ['<special-char>'] * len(''.join(tokens_special))
        
    return tokens

### high freq

### low freq

In [36]:
print("Extracting features from the training data using a sparse vectorizer")

t0 = time()

vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1e-3, tokenizer=split1)
tfidf = vectorizer.fit_transform(X_train)
        
duration = time() - t0

print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
print("n_samples: %d, n_features: %d" % tfidf.shape)
print()

Extracting features from the training data using a sparse vectorizer
done in 3.046775s at 0.523MB/s
n_samples: 34686, n_features: 1046



In [37]:
feature_names = vectorizer.get_feature_names()

In [38]:
# with open('./model/ads-detect-1-20210125.vocab', 'wb') as f:
# with open('./model/ads-detect-1-20210312.vocab', 'wb') as f:
with open('./model/ads-detect-1-20210401.vocab', 'wb') as f:
    pickle.dump(feature_names, f)

In [39]:
def low_freq(x):
    return ['<unk>' if i not in feature_names else i for i in x]
    # return ['[UNK]' if i not in feature_names else i for i in x]

### split 2

In [40]:
def split2(s):
    # return low_freq(split1(s))

    tokens = split1(s)
    # tokens = high_freq(tokens)
    tokens = low_freq(tokens)
    
    return tokens

### split test

In [41]:
data_train['content'].head(10).apply(split2)

0                     [<unk>, <unk>, <unk>, <contact>]
1    [九, ,, <unk>, <unk>, 团, 里, 有, <unk>, 之, 妇, ,, ...
2    [<contact>, 元, 宝, +, <contact>, +, 武, 将, +, 各,...
3                [九, ,, 吃, 里, <unk>, 外, ,, 结, 义, 内, 鬼]
4    [资, 源, 功, 勋, 高, 迁, 微, <contact>, 是, 小, <unk>, ...
5    [收, 魏, 国, 高, 迁, 号, <special-char>, <special-ch...
6                      [出, 资, 源, 有, 需, 要, +, <num-11>]
7    [微, <contact>, 以, 下, 刷, 功, 勋, 私, 聊, 公, 平, 刷, ,...
8    [带, 拉, 量, <unk>, 大, ,, 高, 迁, 功, 勋, ,, 大, 量, 低,...
9    [带, 拉, 量, <unk>, 大, ,, 高, 迁, 功, 勋, ,, 大, 量, 低,...
Name: content, dtype: object